In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing


df = pd.read_csv("input.csv", delimiter=",")
df = df.drop(columns=["name","description","temperament"]) # Retira-se nome, descrição e temperamento, pois são irrelevantes para o processamento
df

,popularity,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,group,grooming_frequency_value,grooming_frequency_category,shedding_value,shedding_category,energy_level_value,energy_level_category,trainability_value,trainability_category,demeanor_value,demeanor_category
0,148,22.86,29.21,3.175147,4.535924,12.0,15.0,Toy Group,0.6,2-3 Times a Week Brushing,0.6,Seasonal,0.6,Regular Exercise,0.8,Easy Training,1.0,Outgoing
1,113,63.50,68.58,22.679619,27.215542,12.0,15.0,Hound Group,0.8,Daily Brushing,0.2,Infrequent,0.8,Energetic,0.2,May be Stubborn,0.2,Aloof/Wary
2,60,58.42,58.42,22.679619,31.751466,11.0,14.0,Terrier Group,0.6,2-3 Times a Week Brushing,0.4,Occasional,0.6,Regular Exercise,1.0,Eager to Please,0.8,Friendly
3,47,60.96,71.12,31.751466,58.967008,10.0,13.0,Working Group,0.8,Daily Brushing,0.6,Seasonal,0.8,Energetic,1.0,Eager to Please,0.6,Alert/Responsive
4,58,58.42,63.50,34.019428,38.555351,10.0,14.0,Working Group,0.6,2-3 Times a Week Brushing,0.6,Seasonal,0.8,Energetic,0.4,Independent,0.8,Friendly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,167,54.61,63.50,20.411657,29.483504,12.0,14.0,Sporting Group,0.2,Occasional Bath/Brush,0.6,Seasonal,0.8,Energetic,0.6,Agreeable,0.6,Alert/Responsive
273,NaN,48.26,63.50,12.700586,27.215542,12.0,15.0,Foundation Stock Service,0.2,Occasional Bath/Brush,0.6,Seasonal,0.8,Energetic,0.4,Independent,0.6,Alert/Responsive
274,140,25.40,58.42,4.535924,24.947580,13.0,18.0,Non-Sporting Group,0.2,Occasional Bath/Brush,0.2,Infrequent,0.8,Energetic,0.6,Agreeable,0.6,Alert/Responsive
275,NaN,53.34,58.42,18.143695,24.947580,10.0,12.0,Foundation Stock Service,0.4,Weekly Brushing,0.6,Seasonal,0.8,Energetic,0.2,May be Stubborn,0.4,Reserved with Strangers


#### Ao ler os dados, é notavel que os valores para: grooming_frequency, shedding, energy_level, trainability e demeanor são equivalentes a sua categoria, logo, pode-se ignorar todas as colunas que terminem com "category" visto que são dados redundantes

In [78]:
df = df.drop(columns=["grooming_frequency_category","shedding_category","energy_level_category","trainability_category","demeanor_category"])
df

,popularity,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,group,grooming_frequency_value,shedding_value,energy_level_value,trainability_value,demeanor_value
0,148,22.86,29.21,3.175147,4.535924,12.0,15.0,Toy Group,0.6,0.6,0.6,0.8,1.0
1,113,63.50,68.58,22.679619,27.215542,12.0,15.0,Hound Group,0.8,0.2,0.8,0.2,0.2
2,60,58.42,58.42,22.679619,31.751466,11.0,14.0,Terrier Group,0.6,0.4,0.6,1.0,0.8
3,47,60.96,71.12,31.751466,58.967008,10.0,13.0,Working Group,0.8,0.6,0.8,1.0,0.6
4,58,58.42,63.50,34.019428,38.555351,10.0,14.0,Working Group,0.6,0.6,0.8,0.4,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,167,54.61,63.50,20.411657,29.483504,12.0,14.0,Sporting Group,0.2,0.6,0.8,0.6,0.6
273,NaN,48.26,63.50,12.700586,27.215542,12.0,15.0,Foundation Stock Service,0.2,0.6,0.8,0.4,0.6
274,140,25.40,58.42,4.535924,24.947580,13.0,18.0,Non-Sporting Group,0.2,0.2,0.8,0.6,0.6
275,NaN,53.34,58.42,18.143695,24.947580,10.0,12.0,Foundation Stock Service,0.4,0.6,0.8,0.2,0.4


#### Devido a quase 30% dos dados estarem sem o atributo "popularity" decidi por não incluir esse atributo

In [79]:
df = df.drop(columns="popularity")
df

,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,group,grooming_frequency_value,shedding_value,energy_level_value,trainability_value,demeanor_value
0,22.86,29.21,3.175147,4.535924,12.0,15.0,Toy Group,0.6,0.6,0.6,0.8,1.0
1,63.50,68.58,22.679619,27.215542,12.0,15.0,Hound Group,0.8,0.2,0.8,0.2,0.2
2,58.42,58.42,22.679619,31.751466,11.0,14.0,Terrier Group,0.6,0.4,0.6,1.0,0.8
3,60.96,71.12,31.751466,58.967008,10.0,13.0,Working Group,0.8,0.6,0.8,1.0,0.6
4,58.42,63.50,34.019428,38.555351,10.0,14.0,Working Group,0.6,0.6,0.8,0.4,0.8
...,...,...,...,...,...,...,...,...,...,...,...,...
272,54.61,63.50,20.411657,29.483504,12.0,14.0,Sporting Group,0.2,0.6,0.8,0.6,0.6
273,48.26,63.50,12.700586,27.215542,12.0,15.0,Foundation Stock Service,0.2,0.6,0.8,0.4,0.6
274,25.40,58.42,4.535924,24.947580,13.0,18.0,Non-Sporting Group,0.2,0.2,0.8,0.6,0.6
275,53.34,58.42,18.143695,24.947580,10.0,12.0,Foundation Stock Service,0.4,0.6,0.8,0.2,0.4


In [80]:
le = preprocessing.LabelEncoder() #transforma atributos qualitativos em quantitativos
df["group"] = le.fit_transform(df["group"])
        
print(df)

     min_height  max_height  min_weight  max_weight  min_expectancy  \
0         22.86       29.21    3.175147    4.535924            12.0   
1         63.50       68.58   22.679619   27.215542            12.0   
2         58.42       58.42   22.679619   31.751466            11.0   
3         60.96       71.12   31.751466   58.967008            10.0   
4         58.42       63.50   34.019428   38.555351            10.0   
..          ...         ...         ...         ...             ...   
272       54.61       63.50   20.411657   29.483504            12.0   
273       48.26       63.50   12.700586   27.215542            12.0   
274       25.40       58.42    4.535924   24.947580            13.0   
275       53.34       58.42   18.143695   24.947580            10.0   
276       17.78       20.32    3.175147    3.175147            11.0   

     max_expectancy  group  grooming_frequency_value  shedding_value  \
0              15.0      7                       0.6             0.6   
1  

#### Plotagem do gráfico de correlação

In [81]:
corr= df.corr()
corr

,min_height,max_height,min_weight,max_weight,min_expectancy,max_expectancy,group,grooming_frequency_value,shedding_value,energy_level_value,trainability_value,demeanor_value
min_height,1.000000,0.935146,0.815263,0.792409,-0.460569,-0.463709,-0.132758,-0.186637,0.285123,0.174194,-0.049236,-0.239074
max_height,0.935146,1.000000,0.793010,0.806405,-0.441227,-0.467058,-0.186988,-0.244199,0.320306,0.198560,-0.051842,-0.294494
min_weight,0.815263,0.793010,1.000000,0.947519,-0.541745,-0.551445,-0.052312,-0.138691,0.284724,-0.072179,-0.132576,-0.276253
max_weight,0.792409,0.806405,0.947519,1.000000,-0.513515,-0.522455,-0.050283,-0.116240,0.281833,-0.076534,-0.075793,-0.273650
min_expectancy,-0.460569,-0.441227,-0.541745,-0.513515,1.000000,0.765434,-0.081257,0.028934,-0.251795,-0.000183,0.091815,0.181565
max_expectancy,-0.463709,-0.467058,-0.551445,-0.522455,0.765434,1.000000,0.006266,0.127438,-0.356471,0.012556,0.094136,0.168534
group,-0.132758,-0.186988,-0.052312,-0.050283,-0.081257,0.006266,1.000000,0.247255,-0.103063,0.000261,0.060739,0.220588
grooming_frequency_value,-0.186637,-0.244199,-0.138691,-0.116240,0.028934,0.127438,0.247255,1.000000,-0.070256,-0.151441,0.098834,0.112278
shedding_value,0.285123,0.320306,0.284724,0.281833,-0.251795,-0.356471,-0.103063,-0.070256,1.000000,0.173437,0.043577,-0.179746
energy_level_value,0.174194,0.198560,-0.072179,-0.076534,-0.000183,0.012556,0.000261,-0.151441,0.173437,1.000000,0.119501,-0.003133


#### Pelo gráfico de correlação, é notável que há uma correlação entre min e max height e weight, assim como min e max expectancy. Irei agrupa-los

In [82]:
# Size define a junção das médias das alturas com a média dos pesos, o peso é em geral menor que a altura,
# então para igualar eu divido em uma parte maior a altura
df["size"] = (df["min_height"] + df["max_height"])/8 + (df["min_weight"] + df["max_weight"])/4
df["avg_expectancy"] = (df["min_expectancy"] + df["max_expectancy"])/2
df = df.drop(columns=["min_height","min_weight","min_expectancy","max_height","max_weight","max_expectancy"])
df

,group,grooming_frequency_value,shedding_value,energy_level_value,trainability_value,demeanor_value,size,avg_expectancy
0,7,0.6,0.6,0.6,0.8,1.0,8.436518,13.5
1,2,0.8,0.2,0.8,0.2,0.2,28.983790,13.5
2,6,0.6,0.4,0.6,1.0,0.8,28.212771,12.5
3,8,0.8,0.6,0.8,1.0,0.6,39.189619,11.5
4,8,0.6,0.6,0.8,0.4,0.8,33.383695,12.0
...,...,...,...,...,...,...,...,...
272,5,0.2,0.6,0.8,0.6,0.6,27.237540,13.0
273,0,0.2,0.6,0.8,0.4,0.6,23.949032,13.5
274,4,0.2,0.2,0.8,0.6,0.6,17.848376,15.5
275,0,0.4,0.6,0.8,0.2,0.4,24.742819,11.0


In [83]:
corr = df.corr()
corr

,group,grooming_frequency_value,shedding_value,energy_level_value,trainability_value,demeanor_value,size,avg_expectancy
group,1.000000,0.247255,-0.103063,0.000261,0.060739,0.220588,-0.092752,-0.037489
grooming_frequency_value,0.247255,1.000000,-0.070256,-0.151441,0.098834,0.112278,-0.164069,0.085514
shedding_value,-0.103063,-0.070256,1.000000,0.173437,0.043577,-0.179746,0.306245,-0.325147
energy_level_value,0.000261,-0.151441,0.173437,1.000000,0.119501,-0.003133,0.010476,0.006894
trainability_value,0.060739,0.098834,0.043577,0.119501,1.000000,0.334465,-0.086809,0.098587
demeanor_value,0.220588,0.112278,-0.179746,-0.003133,0.334465,1.000000,-0.288221,0.185147
size,-0.092752,-0.164069,0.306245,0.010476,-0.086809,-0.288221,1.000000,-0.568398
avg_expectancy,-0.037489,0.085514,-0.325147,0.006894,0.098587,0.185147,-0.568398,1.000000


#### Com os agrupamentos feitos, os atributos tem pouquissima correlação entre si.

In [84]:
print('Valores faltantes:\n', df.isnull().sum())

Valores faltantes:
 group                        0
grooming_frequency_value     7
shedding_value              20
energy_level_value           6
trainability_value          24
demeanor_value              25
size                         2
avg_expectancy               3
dtype: int64


In [92]:
# No conjunto de dados, muitos dos dados que possuem mais de um atributos faltantes, e apesar de ter quase 90 atributos faltantes


df = df.dropna(how ='any') #elimina todas as linhas com dados ausentes

print('Valores faltantes:\n', df.isnull().sum())
df


Valores faltantes:
 group                       0
grooming_frequency_value    0
shedding_value              0
energy_level_value          0
trainability_value          0
demeanor_value              0
size                        0
avg_expectancy              0
dtype: int64


,group,grooming_frequency_value,shedding_value,energy_level_value,trainability_value,demeanor_value,size,avg_expectancy
0,7,0.6,0.6,0.6,0.8,1.0,8.436518,13.5
1,2,0.8,0.2,0.8,0.2,0.2,28.983790,13.5
2,6,0.6,0.4,0.6,1.0,0.8,28.212771,12.5
3,8,0.8,0.6,0.8,1.0,0.6,39.189619,11.5
4,8,0.6,0.6,0.8,0.4,0.8,33.383695,12.0
...,...,...,...,...,...,...,...,...
272,5,0.2,0.6,0.8,0.6,0.6,27.237540,13.0
273,0,0.2,0.6,0.8,0.4,0.6,23.949032,13.5
274,4,0.2,0.2,0.8,0.6,0.6,17.848376,15.5
275,0,0.4,0.6,0.8,0.2,0.4,24.742819,11.0
